# Tutorial 2 - The *lac* System with Spatial Resolution

This is like tutorial in CME tutorial  `lac2stateCME.ipynb`, except we investigate the system with spatial resolution. 

The model presented here can be found in the article: [Noise Contributions in an Inducible Genetic Switch: A Whole-Cell Simulation Study](http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1002010).

In [1]:
import os
import jLM                                  # Set up the Jupyter environment
from jLM.RDME import Sim as RDMESim         # Main simulation class
from jLM.RegionBuilder import RegionBuilder # Deal with the spatial geometry
from lm import IntMpdRdmeSolver             # lm::rdme::IntMpdRdmeSolver
import pyLM.units as unitsconv             # Unit converters
import numpy as np
%matplotlib inline

## 2.1 Initialization and spatial geometry

The workflow is the same as the Tutorial 1, we need to create a RDME simulation object `jLM.RDME.Sim` and define spatial geometry, species with relevant counts, diffusion coefficients and reactions.

In [2]:
cellvol = 1.0e-15                # L=dm^3
simname = "Lac Genetic Switch"
filename = "T2_lac2state.lm "

lattice_spacing = 32e-9            # in m

regionN = "extracellular"
dt = 50e-6                         # in s
# we need to create the LM region with (1.024,1.024,2.048) micron, 
# that can fully contain the E.coli
xdim = int(unitsconv.micron(1.024)/lattice_spacing)
ydim = int(unitsconv.micron(1.024)/lattice_spacing)
zdim = int(unitsconv.micron(2.048)/lattice_spacing)
Ldim = [xdim, ydim, zdim]                   # Has to be multiples of 32

print("dimension of the LM lattice: ", Ldim)

dimension of the LM lattice:  [32, 32, 64]


In [3]:
sim = RDMESim(  simname, 
                filename, 
                Ldim, 
                lattice_spacing, 
                regionN,
                dt)

Now we create a ellipsoid to represent the E.coli.

In [4]:
build = RegionBuilder(sim)
# cytoplasm = build.capsule(length=unitsconv.micron(1.8), width=unitsconv.micron(0.5))
capsule_l = int(unitsconv.micron(1.8)/lattice_spacing)
capsule_w = int(unitsconv.micron(0.5)/lattice_spacing)
print("capsule length: ", capsule_l, "\n capsule width: ", capsule_w)
cytoplasm = build.capsule(length=capsule_l, width=capsule_w)

capsule length:  56 
 capsule width:  15


Now we create the cell membrane for E.coli, which is 64nm.

Consider the lattice_spacing = 32nm, we need 2 cube thick for the membrane.

In [7]:
cyto_dialiation = build.dilate(cytoplasm, radius=int(64/32))
membrane = cyto_dialiation &~ cytoplasm# cyto_di

make extracellualr exlude those regions.

In [8]:
extracellular = ~ (cytoplasm| membrane)

combine all regions into site lattice

In [10]:
cyt = sim.region('cytoplasm')
mem = sim.region('membrane')
ext = sim.region('extracellular')

build.compose(
    (sim.region('extracellular'), extracellular),
    (sim.region('cytoplasm'), cytoplasm),
    (sim.region('membrane'), membrane)
    )

In [15]:
sim.displayGeometry()

Key binding,Function
e,Examine mode
f,Fly mode
w,Walk mode
l,Look mode
r,Reset view
a,Show all
u,Upright
Key binding,Function
Left-click / Shift+Left-click,Rotate
Middle-click / Ctrl+Left-click,Pan


## 2.2 Initialization of Species

Lactose Gene, Product and Inducer Species 
+ O - *lac* operator (e.g. the *lac* gene); it is transcribed to produce mY 
+ mY - mRNA that encodes the lactose permease; it is translated to produce Y
+ Y - lactose permease; it can transport inducer into the cell 
+ I - intracellular inducer (lactose analog) 
+ Iex - extracellular inducer 
+ YI - lactose analog bound by the lactose permease 

Free (cytosolic) Repressor Sepcies
+ R2 - free repressor dimer 
+ IR2 - free repressor with a single inducer bound 
+ I2R2 - free repressor with inducer bound to both dimer subunits 

Repressed Gene Species 
+ R2O - repressed gene with dimer sitting on the operator 
+ IR2O - repressed gene with a single inducer bound to the dimer 
+ I2R2O - repressed gene with inducer molecules bound to both dimer subunits 

In [16]:
species_all = ['R2','O','R2O','IR2','IR2O','I2R2','I2R2O','mY','Y','I','Iex','YI']
with sim.construct():
    for sp in species_all:
        sim.species(sp)

Index,Name
1,R2
2,O
3,R2O
4,IR2
5,IR2O
6,I2R2
7,I2R2O
8,mY
9,Y
10,I


## 2.3 Initialization of reactions

In [28]:
scalar = 2.076e-9 # Rate conversion from experiments to stochastic
sp = sim.sp
rc = sim.rc

### 2.3.1 Lac operon regulation

In [32]:
with sim.construct():
    #sim.rateConst(rate, value, order [, texRepr, annotation])
    sim.rateConst('k1', 2.43e6*scalar,2)
    sim.rateConst('k2', 1.21e6*scalar,2)
    sim.rateConst('k3', 2.43e4*scalar,2)
    sim.rateConst('k4', 6.30e-4     ,1)
    sim.rateConst('k5', 6.30e-4     ,1)
    sim.rateConst('k6', 3.15e-1     ,1)
    
    # region.addReaction(reactants, products, rate)
    cyt.addReaction([sp.R2,sp.O]    ,[sp.R2O]       ,rc.k1)
    cyt.addReaction([sp.IR2,sp.O]   ,[sp.IR2O]      ,rc.k2)
    cyt.addReaction([sp.I2R2,sp.O]  ,[sp.I2R2O]     ,rc.k3)
    cyt.addReaction([sp.R2O]        ,[sp.R2,sp.O]   ,rc.k4)
    cyt.addReaction([sp.IR2O]       ,[sp.IR2,sp.O] ,rc.k5)
    cyt.addReaction([sp.I2R2O]      ,[sp.I2R2,sp.O] ,rc.k6)

Index,Reaction,Rate,Regions
1,$$\mathrm{IR2} + \mathrm{O}\overset{k_{\mathrm{k2}}}{\longrightarrow}\mathrm{IR2O}$$,2.512 × 10-3 M-1⋅s-1,cytoplasm
2,$$\mathrm{I2R2} + \mathrm{O}\overset{k_{\mathrm{k3}}}{\longrightarrow}\mathrm{I2R2O}$$,5.045 × 10-5 M-1⋅s-1,cytoplasm
3,$$\mathrm{R2O}\overset{k_{\mathrm{k4}}}{\longrightarrow}\mathrm{R2} + \mathrm{O}$$,6.300 × 10-4 s-1,cytoplasm
4,$$\mathrm{IR2O}\overset{k_{\mathrm{k5}}}{\longrightarrow}\mathrm{IR2} + \mathrm{O}$$,6.300 × 10-4 s-1,cytoplasm
5,$$\mathrm{I2R2O}\overset{k_{\mathrm{k6}}}{\longrightarrow}\mathrm{I2R2} + \mathrm{O}$$,0.315 s-1,cytoplasm


### 2.3.2 Genetic Information process
Transcription, translation, and degredation

In [ ]:
with sim.construct():
    sim.rateConst('k7', 1.26e-1,1)
    sim.rateConst('k8', 4.44e-2,1)
    sim.rateConst('k9', 1.11e-2,1)
    sim.rateConst('k10', 2.10e-4,1)
    
    cyt.addReaction([sp.O]         ,[sp.O, sp.mY]   ,sc.k7)
    mem.addReaction([sp.mY]        ,[sp.mY, sp.Y]   ,sc.k8)
    cyt.addReaction('mY',         '',           1.11e-2)
    mem.addReaction('mY',         '',           1.11e-2)
    mem.addReaction('Y',          '',           2.10e-4)

### 2.3.3 Inducer-repressor interactions

In [ ]:
cyt.addReaction(('I','R2'),   'IR2',        2.27e4*scalar)
cyt.addReaction(('I','IR2'),  'I2R2',       1.14e4*scalar)
cyt.addReaction(('I','R2O'),  'IR2O',       6.67e2*scalar)
cyt.addReaction(('I','IR2O'), 'I2R2O',      3.33e2*scalar)
cyt.addReaction('IR2',        ('I','R2'),   2.00e-1)
cyt.addReaction('I2R2',       ('I','IR2'),  4.00e-1)
cyt.addReaction('IR2O',       ('I','R2O'),  1.00)
cyt.addReaction('I2R2O',      ('I','IR2O'), 2.00)

### 2.3.4 Inducer Transport

In [ ]:
mem.addReaction('Iex',        'I',          2.33e-3)
mem.addReaction('I',          'Iex',        2.33e-3)
mem.addReaction(('Y','Iex'),  'YI',         3.03e4*scalar)
mem.addReaction('YI',         ('Y','Iex'),  1.20e-1)
mem.addReaction('YI',         ('Y','I'),    1.20e+1)